In [18]:
import Utils as utils
import Collaborative_Filtering_Utils as cf_utils
import pickle

import pandas as pd
import numpy as np
from surprise import SVD, SVDpp, SlopeOne, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import accuracy



In [ ]:
################## Load trained algorithms ##################
'''file = open('/home/Virality_Predictor/models/CF_EN_norm.pkl',"rb")
algo_norm = pickle.load(file)
file.close()

file = open('/home/Virality_Predictor/models/CF_EN_raw.pkl',"rb")
algo_raw = pickle.load(file)
file.close()'''


In [2]:
# Load articles
shared_articles_df = utils.load_shared_articles('shared_articles.csv', ['en'])
article_indices = pd.Series(shared_articles_df.index)


/Users/mirayyuce/Desktop/Projects/Virality_Predictor/Utils.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['domain_name'] = df['url'].apply(split_url)


In [3]:
# Load users with raw virality values
user_interactions_df_raw = utils.load_user_interactions('users_interactions.csv', shared_articles_df, to_normalize=False)

# Loads users with normalized virality
user_interactions_df_norm = utils.load_user_interactions('users_interactions.csv', shared_articles_df, to_normalize=True)

In [4]:
print('Are there duplicate users?')
user_interactions_df_raw['personId'].nunique() != len(user_interactions_df_raw.groupby('personId'))

Are there duplicate users?


False

In [5]:
print('Number of users')
user_interactions_df_raw['personId'].nunique()

Number of users


1644

In [6]:
# Discard users with only one interaction for normalized virality
user_interactions_df_norm = utils.filter_user_interactions(user_interactions_df_norm, 1)

Number of users discarded:  367


In [7]:
# Discard users with only one interaction for raw virality
user_interactions_df_raw = utils.filter_user_interactions(user_interactions_df_raw, 1)

Number of users discarded:  367


In [8]:
# Create train and test datasets for normalized virality
norm_train_data_object, norm_test_data_object, norm_user_interactions_data_object = cf_utils.prepare_train_test_splits_cross_validate(user_interactions_df_norm, is_normalized=True)
norm_built_full_train, norm_anti_test_set_from_train, norm_built_full_test_set, norm_train_to_test_set = cf_utils.prepare_train_test_sets(norm_train_data_object, norm_test_data_object)

normalized_sets = {'cv_train': norm_train_data_object,
                   'cv_test': norm_test_data_object,
                   'k_fold_set': norm_user_interactions_data_object,
                   'trainset': norm_built_full_train,
                   'testset' : norm_built_full_test_set,
                   'trainset_test_it' : norm_train_to_test_set,
                   'anti_trainset': norm_anti_test_set_from_train}


In [9]:
# Create train and test datasets for raw virality
raw_train_data_object, raw_test_data_object, raw_user_interactions_data_object = cf_utils.prepare_train_test_splits_cross_validate(user_interactions_df_raw, is_normalized=False)
raw_built_full_train, raw_anti_test_set_from_train, raw_built_full_test_set, raw_train_to_test_set = cf_utils.prepare_train_test_sets(raw_train_data_object, raw_test_data_object)

raw_sets = {'cv_train': raw_train_data_object,
            'cv_test': raw_test_data_object,
            'k_fold_set': raw_user_interactions_data_object,
            'trainset': raw_built_full_train,
            'testset' : raw_built_full_test_set,
            'trainset_test_it' : raw_train_to_test_set,
            'anti_trainset': raw_anti_test_set_from_train}

In [10]:
# Search for promising algorithm using normalized data
norm_results = cf_utils.algorithm_search(normalized_sets['cv_train'])
norm_results

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity

,test_rmse,fit_time,test_time,test_mse,test_mae
Algorithm,,,,,
BaselineOnly,0.522206,0.062146,0.072562,NaN,NaN
SVDpp,0.522232,50.447872,2.516989,NaN,NaN
SVD,0.529499,5.060994,0.308222,NaN,NaN
KNNWithZScore,0.542969,0.311375,0.554994,NaN,NaN
KNNWithMeans,0.543803,0.163999,0.579196,NaN,NaN
KNNBaseline,0.548493,0.246468,0.897598,NaN,NaN
SlopeOne,0.557965,0.572901,1.498677,NaN,NaN
CoClustering,0.581177,1.435398,0.101659,NaN,NaN
KNNBasic,0.582653,0.096620,0.581674,NaN,NaN


In [11]:
# Search for promising algorithm using raw data

raw_results = cf_utils.algorithm_search(raw_sets['cv_train'])
raw_results

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity

,test_rmse,fit_time,test_time,test_mse,test_mae
Algorithm,,,,,
BaselineOnly,5.817785e+01,0.104083,0.206180,NaN,NaN
KNNWithMeans,6.157958e+01,0.236687,0.775314,NaN,NaN
KNNBasic,7.450375e+01,0.109004,0.454094,NaN,NaN
NormalPredictor,7.951403e+01,0.065711,0.181516,NaN,NaN
SlopeOne,7.968343e+01,0.554531,1.335734,NaN,NaN
KNNWithZScore,7.973102e+01,0.533156,0.787941,NaN,NaN
KNNBaseline,8.800773e+01,0.236131,0.690660,NaN,NaN
CoClustering,1.020473e+02,2.202957,0.104179,NaN,NaN
SVD,9.999263e+06,1.894005,0.178758,NaN,NaN


In [12]:
# Try CoClustering
algorithm = CoClustering

In [13]:
# Grid search parameters for CoClustering

parameters = {'n_cltr_u': [3, 10, 20] ,
              'n_cltr_i': [10, 20, 50],
              'n_epochs': [100, 200, 500],
             }

In [14]:
cf_utils.grid_search(algorithm, parameters, normalized_sets['cv_train'])

<class 'surprise.model_selection.search.GridSearchCV'>
0.5858532493936403
{'n_cltr_u': 3, 'n_cltr_i': 10, 'n_epochs': 200}


,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_cltr_u,param_n_cltr_i,param_n_epochs
0,0.583077,0.597712,0.588815,0.589868,0.006021,4,0.350597,0.370050,0.359552,0.360066,0.007950,11,5.266774,0.668495,0.068555,0.014884,"{'n_cltr_u': 3, 'n_cltr_i': 10, 'n_epochs': 100}",3,10,100
1,0.584695,0.588158,0.584706,0.585853,0.001630,1,0.360929,0.361733,0.355236,0.359299,0.002892,7,11.002448,0.613527,0.113569,0.076818,"{'n_cltr_u': 3, 'n_cltr_i': 10, 'n_epochs': 200}",3,10,200
2,0.583480,0.587456,0.589439,0.586792,0.002478,2,0.352920,0.359798,0.357465,0.356728,0.002856,2,28.973200,2.852454,0.067346,0.003957,"{'n_cltr_u': 3, 'n_cltr_i': 10, 'n_epochs': 500}",3,10,500
3,0.586168,0.597326,0.586471,0.589988,0.005190,5,0.356890,0.369184,0.354351,0.360142,0.006477,12,5.799528,0.093245,0.063257,0.003883,"{'n_cltr_u': 3, 'n_cltr_i': 20, 'n_epochs': 100}",3,20,100
4,0.582375,0.594042,0.592900,0.589773,0.005251,3,0.352844,0.367130,0.362861,0.360945,0.005988,14,11.497691,0.246708,0.105343,0.065917,"{'n_cltr_u': 3, 'n_cltr_i': 20, 'n_epochs': 200}",3,20,200
5,0.586066,0.593739,0.591367,0.590391,0.003208,8,0.356225,0.368626,0.360093,0.361648,0.005180,18,33.735204,1.514524,0.085501,0.016415,"{'n_cltr_u': 3, 'n_cltr_i': 20, 'n_epochs': 500}",3,20,500
6,0.588280,0.597508,0.587656,0.591148,0.004504,9,0.357298,0.365550,0.355571,0.359473,0.004355,10,10.236010,0.102707,0.069878,0.007464,"{'n_cltr_u': 3, 'n_cltr_i': 50, 'n_epochs': 100}",3,50,100
7,0.589631,0.591393,0.590122,0.590382,0.000742,7,0.358988,0.359492,0.364925,0.361135,0.002688,15,21.492832,1.860472,0.631266,0.806890,"{'n_cltr_u': 3, 'n_cltr_i': 50, 'n_epochs': 200}",3,50,200
8,0.583998,0.600730,0.586114,0.590281,0.007439,6,0.353008,0.378406,0.359250,0.363555,0.010806,19,57.825895,6.228116,0.076697,0.014745,"{'n_cltr_u': 3, 'n_cltr_i': 50, 'n_epochs': 500}",3,50,500
9,0.590782,0.609280,0.600764,0.600275,0.007559,13,0.352495,0.366917,0.358725,0.359379,0.005906,9,9.254830,0.223023,0.090219,0.009151,"{'n_cltr_u': 10, 'n_cltr_i': 10, 'n_epochs': 100}",10,10,100


In [15]:
cf_utils.grid_search(algorithm, parameters, raw_sets['cv_train'])

<class 'surprise.model_selection.search.GridSearchCV'>
63.05397876488099
{'n_cltr_u': 3, 'n_cltr_i': 50, 'n_epochs': 200}


,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_cltr_u,param_n_cltr_i,param_n_epochs
0,99.943767,45.016084,64.152480,69.704111,22.765149,21,17.289492,16.556230,18.463061,17.436261,0.785348,22,5.395463,0.311744,0.088918,0.028613,"{'n_cltr_u': 3, 'n_cltr_i': 10, 'n_epochs': 100}",3,10,100
1,99.119214,68.124107,113.324075,93.522465,18.872413,26,16.721542,16.522849,20.754032,17.999474,1.949455,26,10.516042,0.259761,0.062338,0.003412,"{'n_cltr_u': 3, 'n_cltr_i': 10, 'n_epochs': 200}",3,10,200
2,99.058500,44.217216,49.440468,64.238728,24.713464,4,17.179597,15.785274,16.060815,16.341896,0.602931,6,25.257760,0.496139,0.102833,0.057918,"{'n_cltr_u': 3, 'n_cltr_i': 10, 'n_epochs': 500}",3,10,500
3,104.412595,42.937771,48.754424,65.368263,27.710446,6,17.510202,15.408692,15.969456,16.296116,0.888488,5,6.151444,0.086177,0.085235,0.027890,"{'n_cltr_u': 3, 'n_cltr_i': 20, 'n_epochs': 100}",3,20,100
4,106.209097,44.719770,48.740699,66.556522,28.086616,15,18.491235,15.560779,16.261775,16.771263,1.249420,12,12.694044,0.293594,0.118010,0.070664,"{'n_cltr_u': 3, 'n_cltr_i': 20, 'n_epochs': 200}",3,20,200
5,96.697480,44.842970,48.488371,63.342940,23.632128,2,15.709353,16.435545,16.149243,16.098047,0.298669,2,32.078525,1.495418,0.068137,0.001333,"{'n_cltr_u': 3, 'n_cltr_i': 20, 'n_epochs': 500}",3,20,500
6,98.072542,44.408098,48.404151,63.628264,24.410358,3,16.922986,15.448796,15.936191,16.102658,0.613239,3,9.496972,0.436008,0.063997,0.001965,"{'n_cltr_u': 3, 'n_cltr_i': 50, 'n_epochs': 100}",3,50,100
7,97.882874,43.190413,48.088649,63.053979,24.708799,1,16.519142,16.185708,15.506359,16.070403,0.421429,1,18.572121,0.419533,0.128815,0.087563,"{'n_cltr_u': 3, 'n_cltr_i': 50, 'n_epochs': 200}",3,50,200
8,104.366079,42.917096,50.469702,65.917625,27.361447,8,18.148356,15.427394,16.507047,16.694266,1.118689,11,48.038864,1.512691,0.061549,0.001823,"{'n_cltr_u': 3, 'n_cltr_i': 50, 'n_epochs': 500}",3,50,500
9,100.779998,80.703083,167.434075,116.305719,37.070674,27,16.874306,18.582888,26.962927,20.806707,4.408635,27,5.835695,0.277437,0.069413,0.001966,"{'n_cltr_u': 10, 'n_cltr_i': 10, 'n_epochs': 100}",10,10,100


In [16]:
# Train and test the normalized data using the best parameters

algo_norm = CoClustering(n_cltr_u= 3, n_cltr_i= 10, n_epochs= 200)
algo_norm.fit(normalized_sets['trainset'])
predictions_norm_anti_trainset = algo_norm.test(normalized_sets['anti_trainset'])
accuracy.rmse(predictions_norm_anti_trainset)
predictions_norm_testset = algo_norm.test(normalized_sets['testset'])
accuracy.rmse(predictions_norm_testset)

RMSE: 0.3196
RMSE: 0.5802


0.5801758936265226

In [19]:
file = open('/home/Virality_Predictor/models/CF_EN_norm.pkl',"wb")
pickle.dump(algo_norm,file)
file.close()

In [20]:
precision5, recall5 = cf_utils.precision_recall_k_fold(fold=5, at_k=5, thr=0.1, algorithm=algo_norm, complete_data=normalized_sets['k_fold_set'])
print(precision5, recall5)

Precision at 5 0.23990435706695004
Recall at 5 0.21960955156976475
Precision at 5 0.22904812082894285
Recall at 5 0.21041763982582354
Precision at 5 0.2457562568008705
Recall at 5 0.22576628462187667
Precision at 5 0.2577848549186129
Recall at 5 0.22888049793981194
Precision at 5 0.24673063255152813
Recall at 5 0.23170045043376689


(0.24384484443338086, 0.22327488487820876)

In [21]:
precision10, recall10 = cf_utils.precision_recall_k_fold(fold=5, at_k=10, thr=0.1, algorithm=algo_norm, complete_data=normalized_sets['k_fold_set'])
print(precision10, recall10)

Precision at 10 0.24651059918917068
Recall at 10 0.2464949965016505
Precision at 10 0.22751958689458687
Recall at 10 0.2277033780302299
Precision at 10 0.22889485992934275
Recall at 10 0.23352804983679515
Precision at 10 0.22868070554355652
Recall at 10 0.2326450941226753
Precision at 10 0.23311177962905827
Recall at 10 0.24011759586817014


(0.11647175311857152, 0.11804891143595211)

In [22]:
# Top 3 predictions per user
cf_utils.print_top_n_predictions_per_user(predictions_norm_testset, shared_articles_df, article_indices, 3)

User  -5527145562136413747
MIT and Microsoft Research made a 'smart' tattoo that remotely controls your phone on theverge
Meet Bozoma Saint John, the Apple executive who stole the show at WWDC on businessinsider
Defining The Sharing Economy: What Is Collaborative Consumption--And What Isn't? on fastcoexist
  
User  4973223873869255414
The Broken Window Theory on blog.codinghorror
  
User  2062670502532932588
WebAIM: WebAIM's WCAG 2.0 Checklist - for HTML documents on webaim
How Inflation Is Stealing Your Money: Can Cryptocurrency Solve Inflation on a Global Scale? on bitcoinmagazine
Writing good code: how to reduce the cognitive load of your code on chrismm
  
User  2698350543662359063
WeChat's world on economist
  
User  6756039155228175109
Certeza que devemos marcar uma reunião? on atlassian
12 Agile principles on oreilly
Introducing Mini Courses with MS Chatbots on sitepoint
  
User  4531872167062142431
Speed Up a WordPress Website in 8 Steps on joaoperibeiro
How it feels to learn J

Coca-Cola Is Paying $575 Million for Unilever's Soy Drink Business on fortune
  
User  -7487897518634781969
Apple's Machine Learning Has Cut Siri's Error Rate by a Factor of Two on macrumors
JavaScript Conquered the Web. Now It's Taking Over the Desktop on wired
5 reasons your employees aren't sharing their knowledge on justcuriousblog
  
User  1317292353478724560
Dark Scrum on ronjeffries
Google admits original enterprise cloud strategy was wrong, why it's gone in a different direction - TechRepublic on techrepublic
Google lags behind Amazon and Microsoft's cloud in one important area on businessinsider
  
User  7157933165055102118
Building a Team from the Inside Out: Alok Gupta on the Evolution of Data Science at Airbnb on blog.kaggle
  
User  3568240335801676264
How Coca-Cola Is Harvesting Innovation Energy From Startups on forbes
Don't document your code. Code your documentation. on dev
Google Ranking Factors: The Complete List on backlinko
  
User  8841741572929644986
Google's art

User  7022645187549453002
The 100 (TV Series 2014- ) on imdb
'The Simpsons' celebrates 600 episodes with a VR couch gag on engadget
Google Now Posting Full OTA Images For Nexus Devices Alongside Factory Images on androidpolice
  
User  7565903375902761063
Changing change management on mckinsey
  
User  -4951114616026800382
Google's Cloud Investments are Finally Starting to Pay Off on wired
On Orchestrated Microservices - Built to Adapt on medium
10 Modern Software Over-Engineering Mistakes on medium
  
User  -4752363236131440693
Two Different Sales Motions . . . . on linkedin
Nintendo has doubled in value since Pokémon Go's release on theverge
Ray Kurzweil: The world isn't getting worse - our information is getting better on geekwire
  
User  4231150940887368692
Being A Developer After 40 - Free Code Camp on medium.freecodecamp
The Developer's Midlife Crisis: What's Next? on dzone
  
User  -5230721907253934520
Australian Bitcoin Entrepreneur Launches Robo-Advice Platform on bitcoinist


How One Jira Ticket Made My Employer $1MM/Month: 7 Metrics that Actually Matter - JavaScript Scene on medium
From E-Commerce to Web 3.0: Let the Bots Do the Shopping on wired
  
User  5357244621297233547
On Orchestrated Microservices - Built to Adapt on medium
Spring Boot 1.5.1 released on spring
Introducing Kotlin support in Spring Framework 5.0 on spring
  
User  -7180761520671049625
SwiftyDB - An alternative for managing SQLite Databases on iOS on oyvindkg.github
Spanner, the Google Database That Mastered Time, Is Now Open to Everyone on wired
  
User  -3107690258526146180
The future of agriculture - Jim Womack visits a lean farm on planet-lean
#FinTech is all about human-centred design - Chris Skinner's blog on thefinanser.com/2016/06/fintech-human-centred-design
Embracing Agile on hbr
  
User  9038446466275805109
Why You Should Stop Installing Your WebDev Environment Locally - Smashing Magazine on smashingmagazine
  
User  2785512916786763092
Former Google career coach shares a vi

Top 5 Android Testing Frameworks with Examples on testdroid
  
User  5226673542598775232
Don't document your code. Code your documentation. on dev
  
User  1539552466471655556
Why Programmers Want Private Offices on business.stackoverflow
Amazon open-sources its own deep learning software, DSSTNE on venturebeat
  
User  7421498048184238966
The sales secrets of high-growth companies on mckinsey
Amazon has a huge opportunity in grocery delivery on businessinsider
  
User  7438444945156715220
Java 8 Streams - A Deeper Approach About Performance Improvement on guirmendes.blogspot.com
Changing change management on mckinsey
  
User  -5404285628365611524
BDD Best Practices and Guidelines - Testing Excellence on testingexcellence
  
User  -588447608033313001
How to build cross-platform mobile apps using nothing more than a JSON markup on medium.freecodecamp
My favorite people and resources to learn Android programming from on m.signalvnoise
Hype Driven Development on blog.daftcode
  
User  -48

User  -840631936412652041
How To Manage Your Drupal Patches with 'Drush Patch File' | Chromatic on chromatichq
  
User  1355612258071912541
Former Google career coach shares a visual trick for figuring out what to do with your life on cnbc
  
User  -887301601153643466
LATIN INNOVATION BONANZA | New South & Central America Trend Briefing from TrendWatching on trendwatching
  
User  4110069906204072059
The Democratization of Innovation for the Internet of Things on spectrum.ieee
  
User  -3806521031040937656
Java 8 Best Practices Cheat Sheet on zeroturnaround
Former Google career coach shares a visual trick for figuring out what to do with your life on cnbc
  
User  4989759812391042852
New Amazon Linux Container Image for Cloud and On-Premises Workloads on aws.amazon
  
User  9137372837662939523
New blog from Dries: How should you decouple Drupal? on us8.campaign-archive1
NTT to buy Dell's services division for $3.05 billion on techcrunch
  
User  -7377845572432324217
How enterprise arch

In [23]:
algo_raw = CoClustering(n_cltr_u= 3, n_cltr_i= 50, n_epochs= 200)
algo_raw.fit(raw_sets['trainset'])
predictions_raw_anti_trainset = algo_raw.test(raw_sets['anti_trainset'])
accuracy.rmse(predictions_raw_anti_trainset)
predictions_raw_testset = algo_raw.test(raw_sets['testset'])
accuracy.rmse(predictions_raw_testset)

RMSE: 23.5452
RMSE: 32.7478


32.747845512481454

In [24]:
file = open('/home/Virality_Predictor/models/CF_EN_raw.pkl',"wb")
pickle.dump(algo_raw,file)
file.close()

In [25]:
precision5r, recall5r = cf_utils.precision_recall_k_fold(fold=5, at_k=5, thr=50, algorithm=algo_raw, complete_data=raw_sets['k_fold_set'])
print(precision5r, recall5r)


Precision at 5 0.02117456896551724
Recall at 5 0.01961825678547281
Precision at 5 0.019709837225760796
Recall at 5 0.01980925420415866
Precision at 5 0.0148109243697479
Recall at 5 0.013936364019291927
Precision at 5 0.016829887558940874
Recall at 5 0.014069450012408606
Precision at 5 0.021076313894888413
Recall at 5 0.01989869381878021
0.018720306402971044 0.017466403768022443


In [26]:

precision10r, recall10r = cf_utils.precision_recall_k_fold(fold=5, at_k=10, thr=50, algorithm=algo_raw, complete_data=raw_sets['k_fold_set'])
print(precision10r, recall10r)

Precision at 10 0.018066416040100255
Recall at 10 0.01594702442380461
Precision at 10 0.012838717311560761
Recall at 10 0.015109966599348294
Precision at 10 0.01914098972922502
Recall at 10 0.014996368917937543
Precision at 10 0.01984856284856285
Recall at 10 0.02248531648531649
Precision at 10 0.02384621949138078
Recall at 10 0.022060582786389234
0.009374090542082967 0.009059925921279615


In [27]:
# Top 3 predictions per user
cf_utils.print_top_n_predictions_per_user(predictions_raw_testset, shared_articles_df, article_indices, 3)

User  -5527145562136413747
MIT and Microsoft Research made a 'smart' tattoo that remotely controls your phone on theverge
Meet Bozoma Saint John, the Apple executive who stole the show at WWDC on businessinsider
Defining The Sharing Economy: What Is Collaborative Consumption--And What Isn't? on fastcoexist
  
User  4973223873869255414
The Broken Window Theory on blog.codinghorror
  
User  2062670502532932588
How Inflation Is Stealing Your Money: Can Cryptocurrency Solve Inflation on a Global Scale? on bitcoinmagazine
WebAIM: WebAIM's WCAG 2.0 Checklist - for HTML documents on webaim
If women in tech stop apologizing, we might just be able to shut the sexist shit down. - Athena Talks on medium
  
User  2698350543662359063
WeChat's world on economist
  
User  6756039155228175109
Certeza que devemos marcar uma reunião? on atlassian
12 Agile principles on oreilly
On Orchestrated Microservices - Built to Adapt on medium
  
User  4531872167062142431
How it feels to learn Javascript in 2016 o

User  1022806743657391407
PSA: iOS 9.3.5 fixes serious security exploit that exposed contacts, texts, calls, &amp; emails on 9to5mac
  
User  -3474152259371895547
An overview of web service solutions in Drupal 8 on buytaert
Deep-learning algorithm predicts photos' memorability at "near-human" levels on news.mit
Graph Capabilities with the Elastic Stack on elastic
  
User  -8845298781299428018
Why Steve Jobs Was Proud to Steal Ideas on inc
How the Communicator's Dilemma Made a 'Slack-lash' Inevitable on entrepreneur
Hiri wants to fix the workplace email problem on techcrunch
  
User  8164031568920151381
Clean Coder Blog on blog.cleancoder
Anti-If: The missing patterns on code.joejag
"Eat, sleep, code, repeat" is such bullshit - Signal v. Noise on m.signalvnoise
  
User  -3458422503840091409
Digital manufacturing could eliminate warehouses, create on-demand production on cio
Top 15 Facebook Open Source Projects You Must Know on fossbytes
Don't document your code. Code your documentation.

User  -4028919343899978105
Mastering Bash and Terminal on blockloop
How to Build and Publish ES6 Modules Today, with Babel and Rollup on medium
Most Interesting APIs in 2016: Cognitive Computing on programmableweb
  
User  7871236101796363710
Autofill: What web devs should know, but don't on blog.cloudfour
Brazil's Bank Itaú Joins R3 Blockchain Consortium - CoinDesk on coindesk
Spotify UI built with HTML / CSS - Freebiesbug on freebiesbug
  
User  361614354650880527
The Ultimate Guide to SEO Reporting: Starting with the Basics on dashthis
Can Accenture Take Over Advertising? on contently
Ray Kurzweil: The world isn't getting worse - our information is getting better on geekwire
  
User  -4716637009380371036
Cubr on cbarker
  
User  1374824663945909617
Using Microservices Architecture as API Enablement Strategy on linkedin
The Best Linux Distros of 2016 on linux
Four Node.js Gotchas that Operations Teams Should Know about - The New Stack on thenewstack
  
User  -5157376071499052964
Dark

  
User  2146546851324474301
How bot-to-bot could soon replace APIs on venturebeat
Moovit's new Facebook chatbot for transport proves v0 is not going to be good on thenextweb
Adopting Open Source Development Processes in Corporate IT on osdelivers.blackducksoftware
  
User  7193191844620858589
The need to lead in data and analytics on mckinsey
Amazon still crushing cloud competition, says Gartner Magic Quadrant for IaaS - TechRepublic on techrepublic
The brilliant mechanics of Pokémon Go on techcrunch
  
User  7167124757087633794
4 Reasons Managers Should Spend More Time on Coaching on hbr
4 lessons we can learn from Denmark about happiness at work on businessinsider
Globant Acquires L4 to Strengthen its Lead Position in the Digital Services Space on finance.yahoo
  
User  -3011221777935346594
Drupal and ambitious digital experiences on buytaert
  
User  -4659085198634436081
14 Cool Drupal 8 modules for site builders | August 2016 on blog.openlucius
Drupal and ambitious digital experie

User  -8686631410634491662
What do you mean by "Event-Driven"? on martinfowler
  
User  5529670804473959645
Strategyzer | Business Model Generation - Book on businessmodelgeneration
  
User  2446947580409722972
Clean Coder Blog on blog.cleancoder
  
User  8556982196776583357
The Death of Drupal Commerce as an Ecommerce Solution - Third & Grove on medium
  
User  -4805195090397567709
When to make a Git Commit on dev
  
User  3631158903888752348
Former Google career coach shares a visual trick for figuring out what to do with your life on cnbc
  
User  -1792997688627776863
Microsoft and HackerRank bring executable code snippets to Bing's search results pages on techcrunch
  
User  -2457136404620190048
New MacBook Pro is not a Laptop for Developers Anymore on blog.devteam
The technology behind preview photos on code.facebook
  
User  1417592850229997386
The Ten Commandments of Egoless Programming on blog.codinghorror
  
User  -664586119638092697
The Skills You'll Need and the Salary You C

User  -8223836278784533306
15 Interesting JavaScript and CSS Libraries for October 2016 on tutorialzine
  
User  -8411878656309735649
Chrome 53 Beta: Shadow DOM, PaymentRequest, and Android autoplay on blog.chromium
  
User  2965795813240480461
Accenture Launches Content Studio on newsroom.accenture
  
User  3258998879061034753
The Genderbread Person v3 | It's Pronounced Metrosexual on itspronouncedmetrosexual
  
User  5447073013858620840
Learn Hiragana: The Ultimate Guide on tofugu
  
User  -1930650513477268660
Former Google career coach shares a visual trick for figuring out what to do with your life on cnbc
  
User  -7658896905823060250
How it feels to learn Javascript in 2016 on hackernoon
  
User  -5795907895499274955
The future is the trust economy on techcrunch
  
User  -240056303677024244
Don't document your code. Code your documentation. on dev
  
User  -9063420486253202900
Google Cloud Endpoints now generally available: a fast, scalable API gateway on cloudplatform.googleblog